# Exploring the Word Corpus and EDA

With the dataset cleaned and truncated, this notebook dives deeper into the actual dialogue and building the word corpus. A large part of that involves examining stop words and making decisions on which words to drop from the data.

In [1]:
# Load in the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

The next few cells set up the data. For more details refer to notebook \#1. The adjusted data set is named `lines_final`.

In [2]:
lines = pd.read_csv('../data/All-seasons.csv')

lines = lines[lines.Season != 'Season']

In [3]:
lines[['Season', 'Episode']] = lines[['Season', 'Episode']].astype('int64')

In [4]:
support_chars = ['Mr. Garrison', 'Chef', 'Sharon',\
                 'Mr. Mackey', 'Gerald', 'Liane', 'Sheila',\
                 'Stephen', 'Ms. Garrison', 'Mrs. Garrison']

lines.loc[lines.Character.isin(support_chars), 'Character'] = 'Support Character'

In [5]:
final_labels = ['Cartman', 'Stan', 'Kyle', 'Butters', 'Randy', 'Support Character']

lines_final = lines[lines.Character.isin(final_labels)]

### Word Corpus

Here the basic corpus is created. Essentially, all the strings of dialogue from the `Line` column are compiled into one list. Then, regular expressions are used to remove the new line figure `\n` from each string.

In [6]:
import re

corpus = lines_final.Line.tolist()

for line in range(len(corpus)):
    corpus[line] = re.sub('\\n', '', corpus[line].rstrip())
    
corpus[:10]

['You guys, you guys! Chef is going away.',
 'Going away? For how long?',
 'Forever.',
 "I'm sorry boys.",
 "Chef said he's been bored, so he joining a group called the Super Adventure Club.",
 'Wow!',
 'Chef?? What kind of questions do you think adventuring around the world is gonna answer?!',
 "What's the meaning of life? Why are we here?",
 "I hope you're making the right choice.",
 "I'm gonna miss him.  I'm gonna miss Chef and I...and I don't know how to tell him!"]

<b>Average length</b><br>
What is the average document length for each line?

In [7]:
np.mean([len(doc.split()) for doc in corpus])

11.160658881931518

### Tokens

With the corpus created, we can now get even more granular and examine the individual words. First, the contractions are removed. Then, the words from the corpus are compiled into one list and then converted to lowercase with all punctuation removed. Next, the verbs and nouns are lemmatized. And finally, a counter is called and all the words are added to an OrderedDict according to their frequency within the corpus.

In [8]:
import string
import contractions
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

for line in range(len(corpus)):
    corpus[line] = contractions.fix(corpus[line])

words = " ".join(corpus).lower()
words = " ".join(word.strip(string.punctuation) for word in words.split())

In [9]:
lem = WordNetLemmatizer()

word_list = word_tokenize(words)
word_list = [lem.lemmatize(w, pos='v') for w in word_list]
words = ' '.join([lem.lemmatize(w) for w in word_list])

In [10]:
from collections import Counter, OrderedDict

word_counts = Counter(words.split())

token_dict = OrderedDict(word_counts.most_common())

Looking at the OrderedDict shows which words show up the most, indicating likely stop words.

In [11]:
[(k, v) for k, v in token_dict.items()][:250]

[('be', 25112),
 ('you', 14922),
 ('i', 13788),
 ('to', 12042),
 ('the', 10594),
 ('do', 8179),
 ('not', 7950),
 ('it', 7724),
 ('a', 7723),
 ('we', 6360),
 ('that', 6171),
 ('and', 6119),
 ('have', 5864),
 ('go', 4624),
 ('what', 4579),
 ('get', 4073),
 ('of', 4018),
 ('this', 3368),
 ('in', 3308),
 ('my', 3067),
 ('on', 3053),
 ('oh', 3025),
 ('all', 2933),
 ('can', 2923),
 ('just', 2859),
 ('me', 2697),
 ('for', 2547),
 ('no', 2534),
 ('your', 2265),
 ('he', 2196),
 ('know', 2125),
 ('will', 2100),
 ('yeah', 2097),
 ('but', 1994),
 ('with', 1948),
 ('so', 1915),
 ('they', 1908),
 ('dude', 1904),
 ('now', 1878),
 ('well', 1874),
 ("'s", 1858),
 ('guy', 1812),
 ('u', 1806),
 ('come', 1777),
 ('right', 1707),
 ('like', 1665),
 ('out', 1663),
 ('want', 1641),
 ('here', 1624),
 ('there', 1597),
 ('think', 1589),
 ('kyle', 1582),
 ('up', 1471),
 ('about', 1450),
 ('see', 1432),
 ('how', 1319),
 ('say', 1287),
 ('okay', 1279),
 ('uh', 1251),
 ('our', 1240),
 ('make', 1226),
 ('if', 1212),


### Frequency Metrics

Beyond just identifying common words, below are a couple functions to analyze different types of word frequency. The first function compares the total number of times a word appears in the corpus, total frequency, versus the number of documents containing the word, the document frequency.

In [12]:
def freq_compare(word, corpus=corpus):
    '''Takes a word and a word corpus and calculates
    the total word frequency and the number of documents
    containing that word.'''
    
    word = word.lower()
    
    words = " ".join(corpus).lower()
    words = " ".join(word.strip(string.punctuation) for word in words.split())
    
    word_counts = Counter(words.split())
    token_dict = OrderedDict(word_counts.most_common())
    
    total_frequency = token_dict[word]
    
    doc_freq = 0
    
    for line in corpus:
        if word in [token.strip(string.punctuation).lower() for token in line.split()]:
            doc_freq += 1
    
    print('The total frequency of the word \'{}\' is: \t'.format(word), total_frequency)
    print('The number of documents with the word \'{}\': \t'.format(word), doc_freq)

Now a sanity check:

In [13]:
freq_compare('A', ['a', 'a a.', 'b'])

The total frequency of the word 'a' is: 	 3
The number of documents with the word 'a': 	 2


The next function compares the total frequency of a particular word between the different character labels. This is useful for assessing if a particular word has more importance for specific characters. 

In [14]:
def compare_labels(term):
    '''Takes a particular word and calculates
    how often it is used by each character'''
    
    term = term.lower()
    
    count_dict = {'Cartman': 0, 'Stan': 0, 'Kyle': 0, 'Butters': 0,\
                  'Randy': 0, 'Support Character': 0}
    
    for k, v in count_dict.items():
        subset = lines_final[lines_final.Character == k]
        
        corpus = subset.Line.tolist()

        for line in range(len(corpus)):
            corpus[line] = re.sub('\\n', '', corpus[line].rstrip())
        
        words = " ".join(corpus).lower()
        words = " ".join(word.strip(string.punctuation) for word in words.split())
    
        word_counts = Counter(words.split())
        token_dict = OrderedDict(word_counts.most_common())
        
        if term in token_dict.keys():
            count_dict[k] += token_dict[term]
        
        # Now convert to a ratio
        count_dict[k] = round((count_dict[k] / len(subset)), 3)
    
    print('How often the word \'{}\' appears in each class:'.format(term))
    print(count_dict)

<b>Trying the functions: </b><br>
Now that the functions are defined, let's examine a couple words from `token_dict`, the OrderedDict of all words. It helps to keep in mind that there are approximately 36,000 documents in the data, and that Cartman, Stan and Kyle have the majority of the lines.

In [15]:
freq_compare('The')
compare_labels('The')

The total frequency of the word 'the' is: 	 10585
The number of documents with the word 'the': 	 7587
How often the word 'the' appears in each class:
{'Cartman': 0.376, 'Stan': 0.225, 'Kyle': 0.22, 'Butters': 0.262, 'Randy': 0.368, 'Support Character': 0.343}


In [16]:
freq_compare('but')
compare_labels('but')

The total frequency of the word 'but' is: 	 1989
The number of documents with the word 'but': 	 1847
How often the word 'but' appears in each class:
{'Cartman': 0.056, 'Stan': 0.048, 'Kyle': 0.048, 'Butters': 0.083, 'Randy': 0.068, 'Support Character': 0.06}


In [17]:
freq_compare('dude')
compare_labels('dude')

The total frequency of the word 'dude' is: 	 1881
The number of documents with the word 'dude': 	 1833
How often the word 'dude' appears in each class:
{'Cartman': 0.042, 'Stan': 0.111, 'Kyle': 0.086, 'Butters': 0.0, 'Randy': 0.001, 'Support Character': 0.001}


The words 'the' and 'but' are common words and have fairly uniform frequencies. On the other hand, the word 'dude', although nearly as frequent as 'but', is used much more frequently by the first three characters. This shows that it might be a good word to help identify those characters.

<b>Sifting through stopwords</b><br>
With the functions defined, we can now run them over the list of common words to check the frequency comparisons and confirm which ones should be dropped as stopwords. Below, I have defined a small function to combine the two functions from above.

In [18]:
def word_freqs(word):
    freq_compare(word)
    compare_labels(word)

In [19]:
word_freqs("than")

The total frequency of the word 'than' is: 	 299
The number of documents with the word 'than': 	 286
How often the word 'than' appears in each class:
{'Cartman': 0.01, 'Stan': 0.008, 'Kyle': 0.008, 'Butters': 0.007, 'Randy': 0.009, 'Support Character': 0.007}


For this process, I chose to manually check each word in order to really dig into the list and compare each one. Having done so, the final list of stopwords is defined below. While none of the words in the list are too surprising, some of the words I chose to not treat as stopwords may be of interest. For example, I chose to keep 'my' and 'me' because they had slightly inbalanced distributions, which might owe to Cartman's selfish nature. I also chose to keep 'no' and 'not' because those might be useful later when moving beyond a simple bag-of-words model.<br>
<br>
The final list contains 53 words.

In [20]:
stop_words = ['be', 'you', 'i', 'to', 'the', 'do', 'it',\
              'a', 'we', 'that', 'and', 'have', 'go', 'what',\
              'get', 'of', 'this', 'in', 'on', 'all', 'just',\
              'for', 'he', 'know', 'will', 'but', 'with', 'so',\
              'they', 'now', 'well', "'s", 'guy', 'u', 'come',\
              'like', 'there', 'at', 'would', 'who', 'him',\
              'them', 'his', 'thing', 'where', 'should', 'an',\
              'please', 'maybe', 'their', 'even', 'any', 'than']

In [21]:
len(stop_words)

53

Another thing I noticed while testing the stopwords is that the frequencies for Kyle and Stan were consistenly lower than the other characters. I don't know the exact reason for this, but it seems to indicate that the dialogue for those two characters is a bit more unique.<br>
Here are a couple examples:

In [22]:
compare_labels('in')

How often the word 'in' appears in each class:
{'Cartman': 0.116, 'Stan': 0.07, 'Kyle': 0.067, 'Butters': 0.086, 'Randy': 0.114, 'Support Character': 0.112}


In [23]:
compare_labels('will')

How often the word 'will' appears in each class:
{'Cartman': 0.027, 'Stan': 0.013, 'Kyle': 0.014, 'Butters': 0.02, 'Randy': 0.021, 'Support Character': 0.025}


The frequency proportions are lowere for Kyle and Stan. This type of pattern was surprisingly consistent throughout the list of stopwords